In [1]:
import torch
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# reading the dataset and learning about it
def get_data():
    with open('names.txt', 'r') as f:
        names = f.read().splitlines()
    return names
    
words = get_data()
print("number of names:", len(words))
print("shortest name length: ", min(len(w) for w in words))
print("longest name length: ", max(len(w) for w in words))


number of names: 32033
shortest name length:  2
longest name length:  15


In [3]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


# Building the dataset

In [13]:
block_size = 3 #how many characters we consider to predict the next one

def build_dataset(words, blocksize = 3):
    X, Y = [], []

    for w in words:
        context = [0] * 3
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(len(words) * 0.8)
n2 = int(len(words) * 0.9)

Xtr, Ytr = build_dataset(words[:n1]) # 80%
Xval, Yval = build_dataset(words[n1:n2]) #10%
Xte, Yte = build_dataset(words[n2:]) #10%

In [14]:
print(f'Train size :\t\t{len(Ytr)}')
print(f'validation size :\t{len(Yval)}')
print(f'test size :\t\t{len(Yte)}')

Train size :		182541
validation size :	22813
test size :		22792


# MLP

In [15]:
vocab_size = len(chars)
c_dim = 10 #dimensionality of the character embedding vectors
h_size = block_size * c_dim 
n_hidden = 200 #number of neurons in the hidden layer

g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, c_dim),        generator=g)
W1 = torch.randn((h_size, n_hidden),        generator=g)
b1 = torch.randn(n_hidden,                  generator=g)
W2 = torch.randn((n_hidden, vocab_size),    generator=g)
b2 = torch.randn(vocab_size,                generator=g)

parameters = [C, W1, b1, W2, b2]
print(sum(p.nelement() for p in parameters)) # number of parameters in total

for p in parameters:
    p.requires_grad = True

11686


# Optmization